# This is a tutorial

#### Libraries

In [325]:
import pandas as pd
import numpy as np
import seaborn as sns #pip install seaborn
import matplotlib.pyplot as plt
from IPython.display import display, HTML #pip install ipython

#### Step 1: Data Collection
Sources: (for the csv files, check out this repository...)<br>
[1] <br>
[2] <br>
[3] <br>

In [326]:
df_children_work = pd.read_csv('working-children-out-of-school-ages-7-14-vs-hours-worked-by-children-ages-7-14.csv')
df_usa_salary_potential = pd.read_csv('usa_salary_potential.csv')

df_europe_students = pd.read_csv('europe_college_student_data.csv', sep=';')

In [327]:
#If you want to see the whole table without the '...', you can use the following.
#pd.set_option('display.max_columns', None)  # Show all columns
#pd.set_option('display.max_rows', None)     # Show all rows

#If this is too much data, you can reset back to default (recommended).
#pd.reset_option('display.max_columns')
#pd.reset_option('display.max_rows')

def showTable(df, name):
    display(HTML(f"<center><h3>{name}</h3></center>"))
    display(df)

Brief: Children working hours...

In [328]:
showTable(df_children_work, 'Working children out of school vs. hours worked by children (age 7-14)')

,Entity,Code,Year,"Children in employment, work only (% of children in employment, ages 7-14)","Average working hours of children, study and work, ages 7-14 (hours per week)",Population (historical estimates),Continent
0,Abkhazia,OWID_ABK,2015,NaN,NaN,NaN,Asia
1,Afghanistan,AFG,2011,50.0,13.1,29249156.0,NaN
2,Afghanistan,AFG,-10000,NaN,NaN,14737.0,NaN
3,Afghanistan,AFG,-9000,NaN,NaN,20405.0,NaN
4,Afghanistan,AFG,-8000,NaN,NaN,28253.0,NaN
...,...,...,...,...,...,...,...
58142,Zimbabwe,ZWE,2017,NaN,NaN,14751101.0,NaN
58143,Zimbabwe,ZWE,2018,NaN,NaN,15052191.0,NaN
58144,Zimbabwe,ZWE,2019,NaN,NaN,15354606.0,NaN
58145,Zimbabwe,ZWE,2020,NaN,NaN,15669663.0,NaN


Brief: Perhaps money is one thing that attracts people to go to universities. It is one of the motivations that drive students to persue higher education.

In [329]:
showTable(df_usa_salary_potential, 'Potential Salary for College Graduates (USA)')

,rank,name,state_name,early_career_pay,mid_career_pay,make_world_better_percent,stem_percent
0,1,Auburn University,Alabama,54400,104500,51.0,31
1,2,University of Alabama in Huntsville,Alabama,57500,103900,59.0,45
2,3,The University of Alabama,Alabama,52300,97400,50.0,15
3,4,Tuskegee University,Alabama,54500,93500,61.0,30
4,5,Samford University,Alabama,48400,90500,52.0,3
...,...,...,...,...,...,...,...
930,22,Viterbo University,Wisconsin,46800,81900,62.0,3
931,23,Concordia University-Wisconsin,Wisconsin,46700,81600,61.0,9
932,24,University of Wisconsin-Parkside,Wisconsin,46000,81400,47.0,17
933,25,University of Wisconsin-River Falls,Wisconsin,47100,81300,52.0,14


Brief: 

Brief: Europe students...

In [330]:
showTable(df_europe_students, 'Europe College Student Info')

,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,1,1,6,9773,1,1,125.0,1,1,1,...,0,6,8,5,12.666667,0,15.5,2.8,-4.06,Graduate
4420,1,1,2,9773,1,1,120.0,105,1,1,...,0,6,6,2,11.000000,0,11.1,0.6,2.02,Dropout
4421,1,1,1,9500,1,1,154.0,1,37,37,...,0,8,9,1,13.500000,0,13.9,-0.3,0.79,Dropout
4422,1,1,1,9147,1,1,180.0,1,37,37,...,0,5,6,5,12.000000,0,9.4,-0.8,-3.12,Graduate


Brief:

#### Step 2: Data Processing
Many times, the data we got is messy. In this process, we will clean some data. Check for NaN values and duplicate rows first. Drop uneccessary columns, and maybe change the column name if needed.

In [331]:
#df.columns
def peekData(df):
    print('---------------------------------------------------')
    print('Table NaN values count for each column\n') 
    print(df.isna().sum(), '\n')
    print('Table duplicated rows count', df.duplicated().sum(), '\n')
    print('---------------------------------------------------')

In [332]:
peekData(df_children_work)

---------------------------------------------------
Table NaN values count for each column

Entity                                                                               0
Code                                                                              3281
Year                                                                                 0
Children in employment, work only (% of children in employment, ages 7-14)       57865
Average working hours of children, study and work, ages 7-14 (hours per week)    58015
Population (historical estimates)                                                   48
Continent                                                                        57862
dtype: int64 

Table duplicated rows count 0 

---------------------------------------------------


Observed: For the children work dataset, there are quite a lot of NaN values, so doing something like imputation won't work. In this case, we are interested in Entity, year, population, and child work hours. So, we can drop the rows that has NaN values in these columns to then see what result we get. Also, the column name is too long, so we will shorten the name.

In [333]:
#Don't need Continent and Code column, drop it.
df_children_work.drop(columns=['Continent', 'Code'], inplace=True)

df_children_work.rename(
    columns={'Children in employment, work only (% of children in employment, ages 7-14)': 'Work_only_pct', 
             'Average working hours of children, study and work, ages 7-14 (hours per week)': 'Work_study(hrs/week)',
             'Population (historical estimates)' : 'Population'}, 
    inplace=True
)

df_children_work[(df_children_work['Year'] >= 1999) & (df_children_work['Year'] <= 2016)]

df_children_work = df_children_work.dropna(subset=['Work_only_pct', 'Work_study(hrs/week)', 'Population'])

#if you want to double check, uncomment this line.
#peekData(df_children_work) 

df_children_work

,Entity,Year,Work_only_pct,Work_study(hrs/week),Population
1,Afghanistan,2011,50.000000,13.10000,29249156.0
598,Albania,2010,5.100000,13.60000,2913402.0
855,Algeria,2013,4.508655,3.60000,38000628.0
1487,Angola,2001,26.600000,12.50000,16941584.0
2389,Armenia,2010,0.000000,5.40000,2946296.0
...,...,...,...,...,...
56267,Venezuela,2006,19.844778,21.30000,27102084.0
56268,Venezuela,2013,31.134620,16.28939,29838022.0
56526,Vietnam,2012,19.000000,11.60000,89301328.0
57245,Yemen,2010,35.970116,16.90000,24743944.0


In [334]:
peekData(df_usa_salary_potential)

---------------------------------------------------
Table NaN values count for each column

rank                          0
name                          0
state_name                    0
early_career_pay              0
mid_career_pay                0
make_world_better_percent    33
stem_percent                  0
dtype: int64 

Table duplicated rows count 1 

---------------------------------------------------


Observed: For the USA college graduate postential salary, we care about the pay, so we will drop the irrelevant column "make_world_better_percent".

In [335]:
df_usa_salary_potential = df_usa_salary_potential.drop(columns=['make_world_better_percent'])
df_usa_salary_potential = df_usa_salary_potential.drop_duplicates(keep='last') #keep only one of the duplicated rows.
df_usa_salary_potential

,rank,name,state_name,early_career_pay,mid_career_pay,stem_percent
0,1,Auburn University,Alabama,54400,104500,31
1,2,University of Alabama in Huntsville,Alabama,57500,103900,45
2,3,The University of Alabama,Alabama,52300,97400,15
3,4,Tuskegee University,Alabama,54500,93500,30
4,5,Samford University,Alabama,48400,90500,3
...,...,...,...,...,...,...
930,22,Viterbo University,Wisconsin,46800,81900,3
931,23,Concordia University-Wisconsin,Wisconsin,46700,81600,9
932,24,University of Wisconsin-Parkside,Wisconsin,46000,81400,17
933,25,University of Wisconsin-River Falls,Wisconsin,47100,81300,14


In [336]:
peekData(df_europe_students)

---------------------------------------------------
Table NaN values count for each column

Marital status                                    0
Application mode                                  0
Application order                                 0
Course                                            0
Daytime/evening attendance\t                      0
Previous qualification                            0
Previous qualification (grade)                    0
Nacionality                                       0
Mother's qualification                            0
Father's qualification                            0
Mother's occupation                               0
Father's occupation                               0
Admission grade                                   0
Displaced                                         0
Educational special needs                         0
Debtor                                            0
Tuition fees up to date                           0
Gender                  

Observed: Good. For the Europe Student dataset, there is no need to clean the data.  

#### Step 3: Exploratory Data Analysis(EDA) & Visualization

maybe make a general function for bar, scatter plotting, etc?

In [337]:
df_children_work = df_children_work.sort_values(by='Year')
print('Children age 7-14 working years to visualize:', df_children_work['Year'].unique())
display(df_children_work.describe())

Children age 7-14 working years to visualize: [1999 2000 2001 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015
 2016]


,Year,Work_only_pct,Work_study(hrs/week),Population
count,132.000000,132.000000,132.00000,1.320000e+02
mean,2010.515152,21.194017,13.61419,3.247172e+07
std,3.554343,18.363822,7.41143,5.065799e+07
min,1999.000000,0.000000,1.90000,1.019369e+06
25%,2009.000000,6.539222,8.37500,5.986888e+06
50%,2011.000000,17.528625,13.05000,1.237385e+07
75%,2013.000000,32.422263,16.50000,2.939637e+07
max,2016.000000,89.345680,40.30000,2.440162e+08
